# Coupled dFLWL-EFO and GDROM for multi-year operation simulation

In this demo, we implement the coupled dFLWL-EFO and GDROM to multi-year operation simulation (WYs 2015-2019) in Folsom Lake. In each year, dFLWL-EFO is applied to the major flood season (November 18 to February 28/29), while the GDROM model is used to simulate real-world release in the non-flood season (June 1 to November 18). For the late flood season (March 1 to May 31), daily releases are determined by the modified storage regulation curve associated with the experiment settings. 

In multi-year operation simulation, GDROM plays multiple roles as summarized below: 
- During the major flood season, M1 is used to determine daily $R_{1,min}$ to meet various water demand. 
- During the non-flood season, the entire operation rules (two modules & module transition conditions) derived by GDROM are used to determine daily release $R_{1}$.

Basic model settings are consistant with those for implementing dFLWL-EFO for the flood season of WY 2017. The difference is that for multi-year operation simulation, we provide additional rules to deal with extreme droughts. Besides, the modified storage regulation curve is provided to guide late flood season operations.


## import necessary modules

In [ ]:
# modules defined in this study
from utils import bma_module # functions to implement BMA to generate BMA-PDFs of forecasted inflow within the forecast horizon
from utils import efo_model # functions to implement the EFO
from utils import dflwl_model # functions to implement the modified dFLWL model to utilize the BMA-PDFs
from utils import gdrom_folsom # pre-derived operations rules for Folsom Lake via the GDROM 


# other modules used in this notebook
from os import listdir
from os.path import isfile, join
import re
import os

import pandas as pd
import numpy as np
import copy

from matplotlib import pyplot as plt 
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import seaborn as sns

## Model setting
- based on real-world operational constraints & operators' preferences and judgements

In [ ]:
#%%%%%% real-world storage regulation curve (only considering major flood season)
# units: TAF (thousand acre-feet)
# res_S_max, S_old_conser are associated with reservoir attributes, which can be obtained from the real-world rule curve 
# W_N: the storage difference btw. the target water conservation level at the end of flood season and the FLWL (i.e., the upper bound of original water conservation pool); 

res_S_max = 967 # usable storage capacity (i.e., storage corresponding to target water conservation level by the end of flood season)
S_old_conser = 367 # storage corresponding to FLWL
W_N = res_S_max - S_old_conser

In [ ]:
#%%%%%% real-world operational constraints
# units: TAF, 1 cfs = 1.9835 acre-feet/day
# R1, current day (i.e., Stage 1) release in volume
# R1_min will be determined by GDROM (provided later)

I_thresh = 30*1.9835 # inflow threshold to determine if there is ongoing flood at current day
R1_max = 115*1.9835 # normal flood control objective release 

# Ramping rate constraints
R1_increase_limit = 80*1.9835 # assuming I0=0 (I0, inflow rate of the previous day), calculated based on ramping rate constraints-1&2
R1_recession_limit = 60*1.9835 # assuming I0=115 kcfs, calculated based on ramping rate constraints-3

In [ ]:
#%%%%%% customizable parameters in dFLWL-EFO
selected_horizon = 7 # selected forecast horizon
W_max = 400 # the upper bound of carryover capacity (FIRO space) during major flood season, in TAF
W_F = 400 # the upper bound of risk-tolerance based flood control capacity during major flood season, in TAF

########## parameters of the dFLWL model
# w: a weight assigned to flood control objective; (1-w), a weight assigned to water conservation
# m: an conponent to define the shape of the loss function, m>1 to ensure the convexity
w = 0.6
m = 3

# r_alpha: risk tolerance levels used in dFLWL model under a forecast horizon of 7 days 
dFLWL_r_alpha = 0.15

# R2_max: downstream flood conveyance capacity at Stage 2 under a forecast horizon of 7 days 
# R2_max was derived from historical release series (unit: TAF)
R2_max = 487.4

########## parameters of the EFO model
# risk-tolerance curve for flood control releases, which is generated based on the X1=5, X2=0.37 (at day 15)
df_risk_tolerance = pd.read_csv(r'...\data\EFO_risk_tolerance_curve.csv')
EFO_risk_tolerance = df_risk_tolerance['risk tolerance'].tolist()


### additonal parameters and rules for multi-year operation simulations

In [ ]:
#%%%%%% additional parameters for multi-year operation simulation
# a storage threhold (e.g., dead storage) to use user-defined rules under extreme drought conditions
S_drought_critical = 300 # in TAF
# the lower bound of daily required release under operation normal conditions (derived from GDROM)
R1_min_lb = 3.932 # in TAF

# modified storage regulation curve associated with the experiment settings.
df_Wm = pd.read_csv(r'...\data\Modified_rule_curve.csv') 

In [ ]:
#%%%%%%%  additional rules to ensure reservoir storage will not go zero during extreme dry conditions
# Users can adjust the rules below to reflect different operational response to extreme droughts 
def emergency_drought_release(I1, R1_min_lb):    
    if I1 <= R1_min_lb: 
        R1 = I1
    else:
        R1 = R1_min_lb
        
    return R1

## Run the coupled dFLWL-EFO and GDROM for multi-year simulations

In [ ]:
#%%%%%%%%%%% read necessary files
# read the folder that saves daily-updated ensemble streamflow forecasts
daily_ens_folder = "...\\data\\CNRFC_daily_ensemble\\"

# read the calibrated parameters of the BMA model (calibrated using R package 'ensembleBMA' externally)
BMA_parameters = pd.read_csv(r'...\data\BMA_paras_7day_horizon.csv', usecols=[1, 2])
# read the fitted lambda value of the Box-Cox transformation (required in implementing BMA model)
df_lambda = pd.read_csv(r'...\data\Box_Cox_trans_lambda.csv') 
fitted_lambda = df_lambda.loc[df_lambda['n_day']==selected_horizon,:]['fitted_lambda'].values

# read observed operation data
df_obs = pd.read_csv(r'...\data\Folsom_observed_operations.csv')
df_obs['date'] = pd.to_datetime(df_obs['date'])
# convert units: 1 million cubic meter (mcm) = 0.8107144 thousand acre-feet
df_obs.loc[:,'I1'] = df_obs['netinflow-mcm'] * 0.8107144
df_obs.loc[:,'S0_obs'] = df_obs['storage-mcm'] * 0.8107144
df_obs.loc[:,'R1_obs'] = df_obs['outflow-mcm'] * 0.8107144
df_obs['S1_obs'] = df_obs['S0_obs'].shift(-1) # S1: daily ending storage


In [ ]:
#%%%%%%%%%%% 
# specify the study period
starting_date = '2015-10-1'
ending_date = '2019-06-30'
chosen_period = pd.date_range(start=starting_date, end=ending_date)
# specify the number of ensemble members
# note: the number of ensemble members during WYs 2015-2019 is 59, while during WYs 2020-2024 is 39
ensemble_number = 59 

In [ ]:
#%%%%%%%%%%% 
# create a dataframe to save results
df_result = pd.DataFrame(columns  = ['date', 'I1', 'R1_sim', 'W1_sim', 'S1_sim'])

# model initialization
S0 = df_obs.loc[df_obs['date'] == starting_date, 'S0_obs'].values[0] # initial storage of the study period (observed value)
W0 = S0 - S_old_conser # initial carryover storage of the study period
R1_0 = df_obs.loc[df_obs['date'] == starting_date, 'R1_obs'].values[0]

# range of MVs of carryover storage (water conservation benefit)
f1_W1_0 = dflwl_model.f_L1_W1(0,W_N,w,m) # MV of W1 when W1=0
f1_W1_max = dflwl_model.f_L1_W1(W_max,W_N,w,m) # MV of W1 when W1=W_max
print("range of MVs of carryover storage:")
print([f1_W1_0, f1_W1_max])

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for date in chosen_period:
    current_date = date.strftime('%Y-%m-%d')
    print("****"+current_date+"****")
    
    #%%%%%%%%%% Beginning of daily time step %%%%%%%%%%
    # assimilate daily-updated ensemble forecasts (unit: kcfs)
    date_file_name = date.strftime('%Y%m%d')
    if os.path.exists(daily_ens_folder+date_file_name+'.csv'):
        df_ensemble = pd.read_csv(daily_ens_folder+date_file_name+'.csv')
        df_ensemble = df_ensemble.iloc[0:15,1:]*1.9835 # unit coversion - kcfs to TAF
        ensemble_nday_lst = df_ensemble.iloc[1:(selected_horizon+1),:].sum().values.tolist() # forecasted cumulative inflow within selected horizon 
    else:
        print("CNRFC ensemble forecast data is missing for the current date!")
        I1 = df_obs.loc[df_obs['date']==date,'I1'].values[0]
        I1_expected = I1 # use the observed value as predicted value for approximation
    
    # compute forecasted inflow at current day
    I1_expected = np.mean(df_ensemble.iloc[0,1:])
    
    # extract seasonal-varying W_max from the modified storage regulation curve
    day_of_month = date.day
    month_seq = date.month
    DOY = date.dayofyear
    W_max_update = df_Wm[(df_Wm['Month']==month_seq) & (df_Wm['Day']==day_of_month)]['W_max'].values[0]
    daily_S_max = W_max_update + S_old_conser
    
    # %%%%%%%%%%%%%%%%%%%%%%%
    # Scenario 1: current date is within the non-flood season 
    # --> using the GDROM to determine daily release R1
    if (month_seq in [6,7,8,9,10,11]) and (DOY <= 322):
        if S0 <= S_drought_critical: # apply additional rules to address very dry conditions 
            R1 = emergency_drought_release(I1_expected, R1_min_lb)
            R1_min = R1
        else: # apply GDROM under normal conditions
            PDSI_obs = df_obs.loc[df_obs['date']==date,'PDSI'].values[0] # for simplicity, we use observed PDSI as input to GDROM in hindcast experiments
            R1 = gdrom_folsom.GDROM_release_prediction(I1_expected, S0, PDSI_obs)
            R1_min = R1
        
    # %%%%%%%%%%%%%%%%%%%%%%%
    # Scenario 2: current date is within the late-flood season (March, April, May)
    # --> using the updated storage regulation curve to determine daily release R1
    elif month_seq in [3,4,5]:
        # first, compute R1_min based on storage level and the derived GDROM model
        if S0 <= S_drought_critical: # additional rules to address very dry conditions
            R1_min = emergency_drought_release(I1_expected, R1_min_lb)
        else:
            R1_min = gdrom_folsom.GDROM_M1(I1_expected, S0)
        
        # then, using the updated rule curve to determine R1
        if W0 >=  W_max_update:
            R1 = I1_expected + (W0 - W_max_update)
        else:
            R1 = I1_expected - (W_max_update - W0)
            
        # additional rules to ensure the release < inflow in late-flood season
        if R1 >= I_thresh and R1 >= I1_expected:
            R1 = I1_expected
    
    
    # %%%%%%%%%%%%%%%%%%%%%%%
    # Scenario 3: current date is within the major flood season (Nov. 18 - Feb. 28/29)
    # --> using the dFLWL-EFO to determine daily release
    else: 
        # first, compute R1_min based on storage level and the derived GDROM model
        if S0 <= S_drought_critical: # additional rules to address very dry conditions
            R1_min = emergency_drought_release(I1_expected, R1_min_lb)
        else:
            R1_min = gdrom_folsom.GDROM_M1(I1_expected, S0)

        #%%%%%%%%%% determine if current inflow (I_expected) >= the flood threshold (I_thresh) %%%%%%%%%% 
        # if yes, apply EFO for flood control releases
        # if not, apply dFLWL for optimal hedging rules
        W1_opt = -9999

        # %%%%%%%%%% apply dFLWL %%%%%%%%%% 
        if I1_expected < I_thresh: # apply dFLWL
            print("### implement dFLWL for optimal hedging rules ###")
            # generate the BMA-PDF of I2 (in cumulative volume) during the selected forecast horizon
            final_pdf, I2_expected, delta_min, pdf_ra, I2_995, pdf_995 = bma_module.get_BMA_PDFs(current_date, ensemble_nday_lst, BMA_parameters, fitted_lambda, dFLWL_r_alpha)

            # use polynomial regression to approximate a section of the BMA-PDF of I2 
            # in this way, dFLWL can find a numerical solutions where MVs of safety margin and carryover storage are equal
            pdf_ra, pdf_995, reg_coeff = bma_module.get_BMA_PDFs_coeff(final_pdf, R2_max, dFLWL_r_alpha)
            f2_delta_min = w * pdf_ra
            f2_delta_995 = w * pdf_995

            # solve optimal carryover storage and saftey margin using KKT conditions
            W1_opt, delta_opt, case_num = dflwl_model.daily_optimal_KKT(I2_expected, delta_min, f2_delta_min, I2_995, f2_delta_995, 
                                                            reg_coeff, R2_max, W_max, W_N, w, m, f1_W1_0, f1_W1_max)
            print('optimal carryover storage: '+str(W1_opt))

            # determine release decision R1 based on water balance
            R1 = W0 + I1_expected - W1_opt

        # %%%%%%%%%% apply EFO %%%%%%%%%% 
        if W1_opt == 0 or I1_expected >= I_thresh:
            print("### implement EFO for flood control release ###")
            R1 = efo_model.EFO_flood_release(df_ensemble, S0, (W_F+S_old_conser), selected_horizon, EFO_risk_tolerance, ensemble_number, R1_min, R1_max, I1_expected)
            print('R1: '+str(R1))
            
        # *********
        # apply additional rules to adjust the release decisions from dFLWL to reduce release fluctuation
        if W1_opt > 0 and R1 >= I_thresh and I1_expected < I_thresh:
            R1_temp = efo_model.EFO_flood_release(df_ensemble, S0, (W_F+S_old_conser), selected_horizon, EFO_risk_tolerance, ensemble_number, R1_min, R1_max, I1_expected)
            if R1_temp < R1 and R1_temp > I_thresh:
                R1 = R1_temp
            if R1_temp < I_thresh:
                R1 = I_thresh
                
        # check the lower and upper bound of carryover storage W1 during major flood season
        W1_true_temp = W0 + I1_expected - R1
        if W1_true_temp <= 0 and W0 >= 0: # to ensure the carryover storage W1 >= 0
            R1 = W0 + I1_expected - 0
        if W1_true_temp <= 0 and W0 < 0:
            R1 = W0 + I1_expected
        if W1_true_temp > W_max: # to ensure the carryover storage W <= W_max
            R1 = W0 + I1_expected - W_max

  
    # %%%%%%%%%% check other release and storage requirements %%%%%%%%%% 
    # check the lower and upper bound of R1
    if R1 < R1_min:
        R1 = R1_min
    if R1 > R1_max:
        if W0 + I1_expected - R1_max <= W_N:
            R1 = R1_max
        else:
            R1 = W_N - W0 - I1_expected

    # check ramping rate constraints
    if R1_0 == R1_max: # decreasing rate limit
        R1 = R1_recession_limit
    if R1_0 < 30*1.9835 and R1 > R1_increase_limit: #increasing rate limit
        R1 = R1_increase_limit
    
    #%%%%%%%%%% End of daily time step %%%%%%%%%%
    # note: actual (i.e., observed) inflow (I1) is known at the end of a decision time step
    # I1 is used to determine daily ending storage at Stage 1 (current step) associated with R1
    I1 = df_obs.loc[df_obs['date']==date,'I1'].values[0]
    W1_true = W0 + I1 - R1 
    S1_true = W1_true + S_old_conser

    # save results
    df_result.loc[len(df_result)] = [current_date, I1, R1, W1_true, S1_true]
    print("actual inflow, release decision, ending storage:")
    print([I1, R1, S1_true])
    
    # update I1_0, R1_0, S0, W0, which might be used to adjust next-step release decision
    I1_0 = I1 
    R1_0 = R1 
    W0 = W1_true
    S0 = S1_true


    

## 4. Plot results

In [ ]:
df_result['date'] = pd.to_datetime(df_result['date'])
df_result = df_result.merge(df_obs[['date', 'R1_obs', 'S0_obs']], left_on='date', right_on='date')
df_result['S1_obs'] = df_result['S0_obs'].shift(1)

In [ ]:
fig, ax = plt.subplots(2, 1,figsize=(8, 5), sharex=True)
ax[0].plot(df_result['date'], df_result['S1_sim'], color='b', label = 'simulated')
ax[0].plot(df_result['date'], df_result['S1_obs'], color='g', linestyle='-.', label = 'observed')
ax[0].set_ylabel('storage (TAF)',fontweight="bold", fontsize=10)
ax[0].legend(loc='upper left', bbox_to_anchor=(0.28, 1.2), ncol=3, frameon=False)
ax[0].grid(True,which='both',linestyle='--')

ax[1].plot(df_result['date'], df_result['R1_sim'], color='b', label = 'simulated')
ax[1].plot(df_result['date'], df_result['R1_obs'], color='g', linestyle = '-.', label = 'observed')
ax[1].fill_between(df_result["date"], df_result["I1"], 0, color='lightskyblue', alpha=0.5)
ax[1].set_ylabel('release (TAF)', fontweight="bold",fontsize=10)
ax[1].grid(True,which='both',linestyle='--')

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%
legend_elements = [Line2D([0], [0], marker='s', color='none', label='observed 1d-inflow',
                          markerfacecolor='lightskyblue', markeredgecolor='none', markersize=10)]
ax[1].legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(0, 1), ncol=1, fontsize=10, frameon=False)     

#ax[1].legend(loc='upper left', bbox_to_anchor=(0, 1), ncol=1)
ax[1].set_ylim([-5,240])
ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.tight_layout()
